In [ ]:
!nvidia-smi

In [ ]:
!pip install diffusers==0.11.1
!pip install transformers scipy ftfy accelerate

In [ ]:
import torch

from diffusers import StableDiffusionPipeline

In [ ]:
model_id = "gsdf/Counterfeit-V2.5"

In [ ]:
prompt = "((masterpiece, best quality)), detailed, 1girl, pink eyes, long hair, black hair, (grapefruit in front), (temple in the background), sitting, red kimono, light smile, arms behind back, water, sunset, hair ornament, "

In [ ]:
num_inference_steps = 20 
guidance_scale = 7

In [ ]:
image_height = 864
image_width = 512

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

pipe.enable_attention_slicing()
pipe.safety_checker = None

In [ ]:
seeds = []

def get_inputs(batch_size=1, manual_seed=None):   
  generator = torch.Generator("cuda")
  for i in range(batch_size):
    seed = generator.seed() if manual_seed == None else manual_seed
    seeds.append(seed)
    generator = generator.manual_seed(seed)
  prompts = batch_size * [prompt]  

  return {"prompt": prompts, "generator": generator, "guidance_scale": guidance_scale, "num_inference_steps": num_inference_steps, "height": image_height, "width": image_width}                                                                                                                                              

In [ ]:
from PIL import Image                                                                                                                                                                                                                         

def image_grid(imgs, rows=1, cols=1):                                                                                                                                                                                                         
    w = image_width
    h = image_height                                                                                                                                                                                                                      
    grid = Image.new('RGB', size=(cols*w, rows*h))                                                                                                                                                                                            
                                                                                                                                                                                                                                              
    for i, img in enumerate(imgs):                                                                                                                                                                                                            
        grid.paste(img, box=(i%cols*w, i//cols*h))    
                                                                                                                                                                                                
    return grid     

In [ ]:
seeds.clear()

images = pipe(**get_inputs(batch_size=2)).images                                                                                                                                                                                              
image_grid(images, rows=1, cols=2)                

In [ ]:
image_index = 0

seed = seeds[image_index]
print("Seed =", seed)
images[image_index]

In [ ]:
seeds.clear()

images = pipe(**get_inputs(batch_size=4, manual_seed=seed)).images                                                                                                                                                                                              
image_grid(images, rows=1, cols=4)  